# Modeling for Football Data from Transfermarkt
- Kaggle Data URL

    - https://www.kaggle.com/datasets/davidcariboo/player-scores

## 데이터 설명
- Transfermarkt 웹사이트에서 스크레이핑한 각종 축구 데이터 (매주 한번 파일 업데이트)
    - 주요 대회 시즌별 60,000개 이상의 경기
    - 모든 대회의 400개 이상의 클럽들
    - 30,000 이상의 선수들
    - 400,000개 이상의 선수 가치 기록
    - 1,200,000개 이상의 선수 출전 기록

- `csv` 파일 설명
    - `apperances` : 선수 출장 기록
    - `club_games` : 클럽별 경기 홈팀, 어웨이팀 정보
    - `clubs` : 리그별 속해 있는 클럽
    - `competitions` : 대회 정보
    - `game_events` : 경기별 이벤트 정보 (카드, 득점, 어시스트 등)
    - `game_lineups` : 경기별 선수의 선발, 교체 명단 등재 여부
    - `games` : 경기에 대한 정보 (시즌, 라운드, 홈팀, 어웨이팀, 순위)
    - `player_valuations` : Transfermarkt 웹사이트에서 매긴 선수의 가치
    - `players` : 선수에 대한 세부 정보

- 데이터베이스 스키마
    <img src="https://raw.githubusercontent.com/dcaribou/transfermarkt-datasets/master/resources/diagram.svg?sanitize=true" width="1700">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import category_encoders as ce
from sklearn.preprocessing import TargetEncoder





In [2]:
# 데이터 경로
path = "C:/Users/aryij/Documents/DataStudy/DAStudy-sat/Tek/football-data-from-transfermarkt/data/df_eda.csv"
df = pd.read_csv(path)

In [3]:
df.head()

,player_name,start_season,yellow_cards,red_cards,goals,assists,minutes_played,competition_code,date_of_birth,age,position,current_club_name,market_value_per_season,country,foot,height_in_cm
0,Aaron Connolly,2019,0,0,3,2,1261,premier-league,2000-01-28,20,Centre-Forward,Brighton and Hove Albion Football Club,4000000.0,Ireland,right,174.0
1,Aaron Connolly,2020,0,0,2,1,791,premier-league,2000-01-28,21,Centre-Forward,Brighton and Hove Albion Football Club,7000000.0,Ireland,right,174.0
2,Aaron Connolly,2021,0,0,0,0,155,premier-league,2000-01-28,22,Centre-Forward,Brighton and Hove Albion Football Club,6000000.0,Ireland,right,174.0
3,Aaron Cresswell,2019,7,0,3,0,2730,premier-league,1989-12-15,30,Left-Back,West Ham United Football Club,6500000.0,England,left,170.0
4,Aaron Cresswell,2020,3,0,0,8,3172,premier-league,1989-12-15,31,Left-Back,West Ham United Football Club,5000000.0,England,left,170.0


In [4]:
# date_of_birth 열 제거 (age로 대체)
df = df.drop("date_of_birth", axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16001 entries, 0 to 16000
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   player_name              16001 non-null  object 
 1   start_season             16001 non-null  int64  
 2   yellow_cards             16001 non-null  int64  
 3   red_cards                16001 non-null  int64  
 4   goals                    16001 non-null  int64  
 5   assists                  16001 non-null  int64  
 6   minutes_played           16001 non-null  int64  
 7   competition_code         16001 non-null  object 
 8   age                      16001 non-null  int64  
 9   position                 16001 non-null  object 
 10  current_club_name        16001 non-null  object 
 11  market_value_per_season  16001 non-null  float64
 12  country                  16001 non-null  object 
 13  foot                     16001 non-null  object 
 14  height_in_cm          

In [6]:
# 수치형, 범주형 변수 따로 선언
num_cols = df.select_dtypes(include=np.number).columns.tolist()
cat_cols = df.select_dtypes(exclude=np.number).columns.tolist()

In [7]:
print("수치형 변수 :", num_cols)
print("범주형 변수 :", cat_cols)

수치형 변수 : ['start_season', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played', 'age', 'market_value_per_season', 'height_in_cm']
범주형 변수 : ['player_name', 'competition_code', 'position', 'current_club_name', 'country', 'foot']


## 데이터 분리

In [8]:
# target 컬럼 "market_value_per_season"
target = "market_value_per_season"
# 나머지 컬럼 features에 선언
features = df.drop(columns=target).columns

# 학습할 train 데이터 : 19-20 ~ 22-23 시즌 데이터 (4시즌)
train = df[df.start_season != 2023]
# 예측할 test 데이터 : 23-24 시즌 데이터
test = df[df.start_season == 2023]

train.shape, test.shape

((13081, 15), (2920, 15))

In [21]:
# validation 데이터셋 생성
train, val = train_test_split(train, train_size=0.2, random_state=42)

In [24]:
# train 데이터 나눠준다
X_train = train[features]
y_train = train[[target]]

# validation 데이터 나눠준다
X_val = val[features]
y_val = val[[target]]

# test 데이터 나눠준다
X_test = test[features]
y_test = test[[target]]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((2616, 14), (2616, 1), (10465, 14), (10465, 1), (2920, 14), (2920, 1))

## 인코딩

### Target Encoding
- 머신 러닝 모델의 출력이 숫자 (회귀)일 때만 범주형 feature에 사용

- 각 범주에 대한 평균 목표값 계산 → 인코딩 하기 위해 범주형 값이 갖는 범주에 해당하는 백분율 대체
- 각 카테고리의 값을 학습 데이터의 target 값의 평균값으로 설정한다
- 데이터 관측치가 많아야 한다
- 과적합의 원인이 될 수 있다
- references
    - https://conanmoon.medium.com/%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B3%BC%ED%95%99-%EC%9C%A0%EB%A7%9D%EC%A3%BC%EC%9D%98-%EB%A7%A4%EC%9D%BC-%EA%B8%80%EC%93%B0%EA%B8%B0-%EC%9D%BC%EA%B3%B1%EB%B2%88%EC%A7%B8-%EC%9D%BC%EC%9A%94%EC%9D%BC-7a40e7de39d4
    - https://velog.io/@seungwoong12/encoding

In [9]:
# target_encoder = ce.target_encoder.TargetEncoder(cols=["player_name"])
# target_encoder.fit(df["player_name"], df["market_value_per_season"])

In [25]:
# Scikit-learn TargetEncoder 사용
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html
# Scikit-learn 버전 높아야 사용 가능 (업데이트 필요)

te = TargetEncoder(smooth="auto", target_type="continuous")
te.fit(train["player_name"].values.reshape(-1, 1), train["market_value_per_season"])


TargetEncoder(target_type='continuous')

In [11]:
# df['name_target'] = te.transform(df['player_name'].values.reshape(-1,1))

In [12]:
# train 데이터 나눠준다
X_train = train[features]
y_train = train[[target]]

# test 데이터 나눠준다
X_test = test[features]
y_test = test[[target]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13081, 14), (2920, 14), (13081, 1), (2920, 1))

In [13]:
# validation 데이터셋 생성
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10464, 14), (2617, 14), (10464, 1), (2617, 1))

In [14]:
# # Scikit-learn TargetEncoder 사용
# # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html

# te = TargetEncoder(smooth="auto", target_type="continuous")
# te.fit(X_train["player_name"].values.reshape(-1, 1), y_train["market_value_per_season"])

In [15]:
# X_train['name_target'] = te.transform(df['player_name'].values.reshape(-1,1))

In [16]:
te = TargetEncoder()
x_encoded = te.fit_transform(X_train, y_train)


c:\Users\aryij\anaconda3\envs\football\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [17]:
x_encoded = pd.DataFrame(x_encoded, columns=te.classes_)
data_target = pd.concat([x_train, x_encoded], axis=1)

ValueError: Shape of passed values is (10464, 2100), indices imply (10464, 150)

## Baseline 모델링